**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Oct 30 04:42:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'AutoCrat Demo Folder'	    "Certify'em Records.gsheet"   yolov3
'AutoCrat Demo Folder (1)'  'Colab Notebooks'
 Certificate.gslides	    'My Drive'


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14358, done.
remote: Total 14358 (delta 0), reused 0 (delta 0), pack-reused 14358
Receiving objects: 100% (14358/14358), 13.06 MiB | 22.67 MiB/s, done.
Resolving deltas: 100% (9765/9765), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "figure" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-10-29 12:56:19--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   111KB/s    in 20m 9s  

2020-10-29 13:16:30 (131 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/PMC449901_00000.jpg  
  inflating: data/obj/PMC449901_00000.txt  
  inflating: data/obj/PMC509421_00001.jpg  
  inflating: data/obj/PMC509421_00001.txt  
  inflating: data/obj/PMC514527_00001.jpg  
  inflating: data/obj/PMC514527_00001.txt  
  inflating: data/obj/PMC514606_00000.jpg  
  inflating: data/obj/PMC514606_00000.txt  
  inflating: data/obj/PMC514606_00001.jpg  
  inflating: data/obj/PMC514606_00001.txt  
  inflating: data/obj/PMC514715_00003.jpg  
  inflating: data/obj/PMC514715_00003.txt  
  inflating: data/obj/PMC514717_00001.jpg  
  inflating: data/obj/PMC514717_00001.txt  
  inflating: data/obj/PMC514891_00002.jpg  
  inflating: data/obj/PMC514891_00002.txt  
  inflating: data/obj/PMC514911_00000.jpg  
  inflating: data/obj/PMC514911_00000.txt  
  inflating: data/obj/PMC516019_00002.jpg  
  inflating: data/obj/PMC516019_00002.txt  
  inflating: data/obj/PMC516042_00001.jpg  
  inflating: data/obj/PMC516042_00001.t

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

16 0.499183 0.669192 0.851307 0.535354

['16', '0.499183', '0.669192', '0.851307', '0.535354']
0 data/obj/PMC544945_00000.txt
0 0.499183 0.669192 0.851307 0.535354
16 0.497549 0.518308 0.834967 0.849747

['16', '0.497549', '0.518308', '0.834967', '0.849747']
1 data/obj/PMC544955_00002.txt
0 0.497549 0.518308 0.834967 0.849747
16 0.207071 0.562580 0.272727 0.816092

['16', '0.207071', '0.562580', '0.272727', '0.816092']
2 data/obj/PMC449901_00000.txt
0 0.207071 0.562580 0.272727 0.816092
16 0.802189 0.563857 0.308081 0.808429

['16', '0.802189', '0.563857', '0.308081', '0.808429']
2 data/obj/PMC449901_00000.txt
0 0.802189 0.563857 0.308081 0.808429
15 0.499158 0.523627 0.281145 0.574713

['15', '0.499158', '0.523627', '0.281145', '0.574713']
2 data/obj/PMC449901_00000.txt
0 0.499158 0.523627 0.281145 0.574713
16 0.499183 0.517677 0.831699 0.840909

['16', '0.499183', '0.517677', '0.831699', '0.840909']
3 data/obj/PMC514527_00001.txt
0 0.499183 0.517677 0.831699 0.840909
15 0.499183 0.44

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/PMC520827_00000.jpg', 'data/obj/PMC1097751_00004.jpg', 'data/obj/PMC545955_00001.jpg', 'data/obj/PMC555764_00004.jpg', 'data/obj/PMC519033_00005.jpg', 'data/obj/PMC1064093_00000.jpg', 'data/obj/PMC548678_00004.jpg', 'data/obj/PMC1064108_00002.jpg', 'data/obj/PMC519031_00005.jpg', 'data/obj/PMC544837_00000.jpg', 'data/obj/PMC1079889_00002.jpg', 'data/obj/PMC522826_00007.jpg', 'data/obj/PMC516042_00006.jpg', 'data/obj/PMC514527_00001.jpg', 'data/obj/PMC548514_00000.jpg', 'data/obj/PMC546202_00005.jpg', 'data/obj/PMC546008_00002.jpg', 'data/obj/PMC1087862_00002.jpg', 'data/obj/PMC555551_00002.jpg', 'data/obj/PMC1087869_00000.jpg', 'data/obj/PMC514606_00001.jpg', 'data/obj/PMC524353_00000.jpg', 'data/obj/PMC1097758_00000.jpg', 'data/obj/PMC554756_00000.jpg', 'data/obj/PMC1064100_00000.jpg', 'data/obj/PMC544940_00000.jpg', 'data/obj/PMC544864_00002.jpg', 'data/obj/PMC551614_00001.jpg', 'data/obj/PMC1084248_00003.jpg', 'data/obj/PMC544945_00000.jpg', 'data/obj/PMC548674_00001.jpg'

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000002, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 192945, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.877899, GIOU: 0.876238), Class: 0.991403, Obj: 0.798982, No Obj: 0.005135, .5R: 1.000000, .75R: 1.000000, count: 5, class_loss = 0.247910, iou_loss = 0.074824, total_loss = 0.322734 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj:

#We have attained approx result so that the training has been stopped after 2000 iterations 